In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import tensorflow as tf
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

2018-08-27 16:50:14.687 INFO in 'summarizer.preprocessing.cleaner'['textcleaner'] at line 37: 'pattern' package not found; tag filters are not available for English
2018-08-27 16:50:14.693 DEBUG in 'matplotlib.backends'['__init__'] at line 90: backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *

### Configuration parameters

In [3]:
NE_TYPES = ['PERSON']
# NE_TYPES = ['PERSON', 'GPE', 'ORG', 'LOC']
# NE_TYPES = ['GPE','DATE','ORG','EVENT','LOC','FAC','CARDINAL','QUANTITY','NORP','ORDINAL','WORK_OF_ART','PERSON']
# NE_TYPES = ['EVENT','LOC','FAC','CARDINAL','QUANTITY','NORP','ORDINAL','WORK_OF_ART','PERSON']
N_EXAMPLES = 1
N_TEST_SENTENCES = 100
N_SAMPLES = 10
MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
# MODELS = ['ne_centroid', 'ne_nearest']
MODELS = ['ne_centroid', 'weighted_kNN', 'svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 2, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}
USE_CV = False

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=True, use_glove=True, elmo_params={'trainable_cells': True, 'restore_path': "./tmp/elmo.ckpt"})

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-27 16:50:15.739 INFO in 'tensorflow'['tf_logging'] at line 159: Using /tmp/tfhub_modules to cache modules.


Restoring finetuned ELMo params from ./tmp/elmo.ckpt
INFO:tensorflow:Restoring parameters from ./tmp/elmo.ckpt


2018-08-27 16:50:16.516 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from ./tmp/elmo.ckpt
2018-08-27 16:50:16.755 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 96: [loading embeddings from `/home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-08-27 16:50:16.755 INFO in 'gensim.models.utils_any2vec'['utils_any2vec'] at line 170: loading projection weights from /home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-08-27 16:50:16.755 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-08-27 16:50:39.228 INFO in 'gensim.models.utils_any2vec'['utils_any2vec'] at line 232: loaded (400000, 100) matrix from /home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-27 16:50:39.418 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


In [6]:
from collections import defaultdict
f1_scores = {}
th_list = {}
for ne_type in NE_TYPES:
    print('NE type: {}'.format(ne_type))
    dataset = filter_dataset_by_ne_types(dataset_orig, ne_type, preserveBIO=True)
    f1_scores[ne_type] = {}
    th_list[ne_type] = {'ne_centroid':[], 'ne_nearest':[]}
    for model in MODELS:
        f1_scores[ne_type][model] = {'values':[], 'values_chunk':[]}
#     f1_scores[ne_type]['avg'] = {'values':[]}'ne_centroid', 'ne_nearest'
    for idx_sample in range(N_SAMPLES):
        print('Sample {}'.format(idx_sample+1))
        tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
        tags_train = removeBIOFromTags(tags_train)
        tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))
        tags_test_bio = tags_test
        tags_test_bio_flat = flatten_list(tags_test_bio)
        tags_test = removeBIOFromTags(tags_test)
        y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
        ner_bin_clf = FewshotNerBinaryClassifier(embedder)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        if USE_CV:
            cv_res = select_sim_thresholds_with_cv(tokens_train,tags_train, embedder)
        pred_avg = np.zeros(y_test.size)
        n_models_avg = 0
        for model in MODELS:
            if model in ['ne_centroid', 'ne_nearest'] and USE_CV:
                pred= pred_class_labels_bin(flat_sim_one_type(results[model]['sim'], sim_type='cosine'), cv_res[model]['threshold'])
                th_list[ne_type][model].append(cv_res[model]['threshold'])
            else:
                pred = results[model]['pred']
            pred_tags_bio = predToTags(pred, accountBIO=True)
            if model in ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']:   
                pred_avg += pred
                n_models_avg += 1
            f1 = f1_score(y_test, pred)
#             print(tags_test_bio)
#             print(pred_tags_bio)
            f1_chunk = precision_recall_f1_chunks(tags_test_bio_flat, pred_tags_bio, print_results=False)['__total__']['f1']/100
            f1_scores[ne_type][model]['values'].append(f1)
            f1_scores[ne_type][model]['values_chunk'].append(f1_chunk)
        pred_avg /= n_models_avg
#         pred_avg = np.array(map(lambda x: 1 if x >= 0.5 else 0, pred_avg))
#         f1 = f1_score(y_test, pred_avg)
#         f1_scores[ne_type]['avg']['values'].append(f1)
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 scores: {}'.format(cur_f1_scores_dict))
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values_chunk'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 chunk scores: {}'.format(cur_f1_scores_dict))                                          
    for model in MODELS:
        f1_scores[ne_type][model]['mean'] = np.mean(f1_scores[ne_type][model]['values'])
        f1_scores[ne_type][model]['std'] = np.std(f1_scores[ne_type][model]['values'])
        print('{} F1: mean = {} std = {}'.format(model, f1_scores[ne_type][model]['mean'], f1_scores[ne_type][model]['std']))
        f1_scores[ne_type][model]['mean_chunk'] = np.mean(f1_scores[ne_type][model]['values_chunk'])
        f1_scores[ne_type][model]['std_chunk'] = np.std(f1_scores[ne_type][model]['values_chunk'])
        if model in ['ne_centroid', 'ne_nearest'] and USE_CV:
            th_list[ne_type][model] = np.mean(th_list[ne_type][model])

NE type: PERSON
Num of train sentences: 12195
Num of valid sentences: 1553
Num of test sentences: 1573
Sample 1
Train
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-27 16:50:42.689 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-27 16:50:43.573 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


NE centroid similarity model
SVM classifier model
Weighted kNN model
k = 2, metric: cosine
[====================] 100%
F1 scores: {'ne_centroid': 0.7380281690140846, 'weighted_kNN': 0.6935779816513762, 'svm': 0.6412698412698412}
F1 chunk scores: {'ne_centroid': 0.6434782608695652, 'weighted_kNN': 0.5149700598802396, 'svm': 0.6057692307692308}
Sample 2
Train
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

NE centroid similarity model
SVM classifier model
Weighted kNN model
k = 2, metric: cosine
[====================] 100%
F1 scores: {'ne_centroid': 0.6732026143790849, 'weighted_kNN': 0.6796116504854369, 'svm': 0.37398373983739835}
F1 chunk scores: {'ne_centroid': 0.4820512820512821, 'weighted_kNN': 0.35454545454545455, 'svm': 0.2666666666666667}
Sample 10
Train
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-27 16:51:37.548 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-27 16:51:38.836 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


NE centroid similarity model
SVM classifier model
Weighted kNN model
k = 2, metric: cosine
[====================] 100%
F1 scores: {'ne_centroid': 0.8175182481751825, 'weighted_kNN': 0.8432203389830508, 'svm': 0.8525345622119815}
F1 chunk scores: {'ne_centroid': 0.6192468619246863, 'weighted_kNN': 0.5683453237410072, 'svm': 0.6718749999999999}
ne_centroid F1: mean = 0.6407056757247496 std = 0.1818371261797493
weighted_kNN F1: mean = 0.7561294938036575 std = 0.0716223081964162
svm F1: mean = 0.47396991719223297 std = 0.2644456459579353


### Print results

In [7]:
print('{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}'.format('MODEL', 'MEAN', 'STD', 'MEAN_CHUNK', 'STD_CHUNK'))
for ne_type in NE_TYPES:
    print('---------------{}---------------'.format(ne_type))
    for model in MODELS:
        print('{:<8s}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}'.format(model, f1_scores[ne_type][model]['mean']*100, f1_scores[ne_type][model]['std']*100,
                                                f1_scores[ne_type][model]['mean_chunk']*100, f1_scores[ne_type][model]['std_chunk']*100))

MODEL   	MEAN    	STD     	MEAN_CHUNK	STD_CHUNK
---------------PERSON---------------
ne_centroid	64.07   	18.18   	46.63   	13.45   
weighted_kNN	75.61   	7.16    	52.96   	11.80   
svm     	47.40   	26.44   	36.13   	22.98   


In [8]:
if USE_CV:
    print('Thresholds chosen with cross-validation:')
    print(th_list)